In [29]:
import os
import json
import itertools
DIR = "../optimization/"
FILENAMES = {    
        "optipc": "02-opt-evaluation-ipc-coverage.json",
        "opt14": "03-opt-evaluation-new2014-coverage.json",
        "opt20": "04-opt-evaluation-new2020-coverage.json",
        "satipc": "05-sat-evaluation-ipc-coverage.json",
        "sat14": "06-sat-evaluation-new2014-coverage.json",
        "sat20": "07-sat-evaluation-new2020-coverage.json",
    }

DOMAINS = [
    "barman", "blocksworld", "childsnack", "data-network", "depot", "driverlog",
    "elevators", "floortile", "grid", "gripper", "hiking", "logistics",
    "miconic", "nomystery", "openstacks", "parking", "rovers", "satellite",
    "scanalyzer", "snake", "storage", "tpp", "transport", "visitall",
    "woodworking", "zenotravel",
]

PLANNERS = {
        "ipc2018-opt-complementary2-3584mb-1800s",
        "ipc2018-opt-decstar",
        "ipc2018-opt-delfi-blind",
        "ipc2018-opt-delfi-celmcut",
        "ipc2018-opt-delfi-ipdb-1800s",
        "ipc2018-opt-delfi-mas-miasm",
        "ipc2018-opt-delfi-mas-sccdfp-1800s",
        "ipc2018-opt-scorpion",
        "ipc2018-agl-decstar-agl",
        "ipc2018-agl-fd-remix",
        "ipc2018-agl-lapkt-bfws-pref",
        "ipc2018-agl-lapkt-dual-bfws",
        "ipc2018-agl-lapkt-poly-bfws",
        "ipc2018-agl-olcff",
        "ipc2018-agl-saarplan",

    }

SHORT_PLANNER_NAME = {
        "ipc2018-opt-complementary2-3584mb-1800s" :             "cmp2",              
        "ipc2018-opt-decstar" :                         "decstar",           
        "ipc2018-opt-delfi-blind" :                     "delfi-blind",       
        "ipc2018-opt-delfi-celmcut" :                   "delfi-celmcut",     
        "ipc2018-opt-delfi-ipdb-1800s" :                "delfi-ipdb",        
        "ipc2018-opt-delfi-mas-miasm" :                 "delfi-miasm",       
        "ipc2018-opt-delfi-mas-sccdfp-1800s" :          "delfi-sccdfp",      
        "ipc2018-opt-scorpion" :                        "scorpion",          
        "ipc2018-agl-decstar-agl" :                     "decstar",           
        "ipc2018-agl-fd-remix" :                        "fd-remix",          
        "ipc2018-agl-lapkt-bfws-pref" :                 "lapkt-bfws",        
        "ipc2018-agl-lapkt-dual-bfws" :                 "lapkt-dual",        
        "ipc2018-agl-lapkt-poly-bfws" :                 "lapkt-poly",        
        "ipc2018-agl-olcff" :                           "olcff",             
"ipc2018-agl-saarplan" :                        "saarplan",
    }


comparisons = {}
COVERAGE = {}

for name, filename in FILENAMES.items():
    with open(os.path.join(DIR, "results", filename)) as f:
        data = json.load(f)
        
        for domain in DOMAINS: 
            coverage = data[domain]
            COVERAGE[(name, domain)]= coverage
            comparisons[(name, domain)] = sum ([1 for (p1, p2) in itertools.combinations(coverage.keys(), 2) if p1 in PLANNERS and p2 in PLANNERS and coverage[p1] != coverage[p2]])
            
    # Only keep the data for a subset of planners and domains.
    #filtered_dict = {}
    #for domain in DOMAINS:
    #    domain_dict = {planner: coverage for planner, coverage in d[domain].items() if planner in PLANNERS}
    #    filtered_dict[domain] = domain_dict
    #dicts[name] = filtered_dict

#algo_dicts = []
#for domain_dict in dicts.values():
#    algo_dicts.extend(domain_dict.values())
    
#dicts



In [41]:
LOGDIR = "../logfiles/2020-09-03/"
import os
import re
import pandas as pd

from IPython.display import display

#2020-11-17 22:58:38,135 INFO     Different evaluated sequences: 202
#2020-11-17 22:58:38,222 INFO     Candidate sequences: 100
#2020-11-17 22:58:38,480 INFO     Solution value  = 0.0

ATRIBUTES_TABLE = ["track", "domain", "evaluated_sequences", "candidate_sequences", "solution_value", "sart_runtimes", "num_sequences", "min_penalty", "max_penalty", "comparisons", "comparisonsipc"]

regex_int = [re.compile(x) for x in [".*Different evaluated sequences: (?P<evaluated_sequences>(\d+))",
                                     ".*Candidate sequences: (?P<candidate_sequences>(\d+))", 
                                    ]]
regex_float = [re.compile(x) for x in [".*Solution value  = (?P<solution_value>(.*))",]]
DATA_LOGS = []
table = []

for logfile in sorted(os.listdir(LOGDIR)):
    _, track, domain = logfile.split("_")
    f = open(f"{LOGDIR}/{logfile}")
    content = f.readlines()
    penalties = [float(l.split(":")[-1]) for l in content if l.startswith("Penalty: ")]
    
    data = { "track" : track, "domain" : domain, "sart_runtimes" : False, "num_sequences" : 0, "min_penalty" : min(penalties) if penalties else "-", 
            "max_penalty" : max(penalties) if penalties else "-", 
            "comparisons" : comparisons[(track, domain.replace('depots','depot').replace('rover','rovers').replace('-strips',''))], 
            "comparisonsipc" : comparisons[(track, domain.replace('depots','depot').replace('rover','rovers').replace('-strips',''))] - comparisons[(track.replace('14','ipc').replace('20','ipc'), domain.replace('depots','depot').replace('rover','rovers').replace('-strips',''))]}
            
    for l in content: 
        if "Using sart runtimes on CPLEX optimization" in l: 
            data["sart_runtimes"] = True
            continue
        if "Selected: sequence" in l:
            data["num_sequences"] += 1

            
        for reg in regex_int: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = int(value)
        
        for reg in regex_float: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = float(value)

    DATA_LOGS.append(data)

    if data["comparisonsipc"]<0:     
        table.append([data[atr] for atr in ATRIBUTES_TABLE])

    
#print("\n".join([f'{d["track"]} {d["domain"]}' for d in DATA_LOGS if not d["sart_runtimes"]]))
pd.options.display.float_format = "{:,.2g}".format
pd.set_option('display.max_rows', 500)

df = pd.DataFrame(table,columns =  ATRIBUTES_TABLE)
display(df)


                 

,track,domain,evaluated_sequences,candidate_sequences,solution_value,sart_runtimes,num_sequences,min_penalty,max_penalty,comparisons,comparisonsipc
0,opt14,data-network,251,180,18,True,2,-,-,25,-2
1,opt14,hiking,438,56,0,True,3,-,-,15,-8
2,opt14,openstacks,722,390,0,True,2,-,-,18,-6
3,opt14,parking,30,4,71,True,3,-,-,24,-4
4,opt14,rover,843,100,12,True,3,-,-,22,-3
5,opt14,scanalyzer,18,7,2.5e+02,True,2,-,-,24,-3
6,opt14,transport,139,99,0,True,2,-,-,18,-3
7,opt20,depots,322,100,16,True,3,-,-,24,-2
8,opt20,elevators,252,80,17,True,3,-,-,23,-3
9,opt20,logistics,97,91,16,True,1,-,-,25,-2


# Analysis of negative results
The following code analyzes in which cases the differences observed in the IPC results does not happen in our new opt20 and sat20 benchmark set.

In [54]:

    
for data in sorted(DATA_LOGS,key=lambda x : x['comparisonsipc']): 
    #if data['comparisonsipc'] < 10:
        
        track, domain = data['track'], data['domain'].replace('rover','rovers').replace('depots','depot').replace('miconic-strips','miconic')
        trackipc = track.replace('14','ipc').replace('20','ipc')
        if '14' in track: 
            continue
        
        cov = COVERAGE[(track, domain)]
        covipc = COVERAGE[(trackipc, domain)]
        
        cmp = sorted([(p1, p2) if covipc[p1] > covipc[p2] else (p2,p1) for (p1, p2) in itertools.combinations(PLANNERS, 2) if p1 in cov and p2 in cov and covipc[p1] != covipc[p2] and cov[p1] == cov[p2]])
        
        cmp += sorted([(p1, p2) if covipc[p1] > covipc[p2] else (p2,p1) for (p1, p2) in itertools.combinations(PLANNERS, 2) if p1 in cov and p2 in cov and (covipc[p1]-covipc[p2])*(cov[p1]-cov[p2]) < 0])
        
        
        if cmp: 
            cmpstr = '\n'.join([f"       {SHORT_PLANNER_NAME[p1]} > {SHORT_PLANNER_NAME[p2]}    {covipc[p1]} {covipc[p2]} -> {cov[p1]} {cov[p2]}" for (p1, p2) in cmp])
            print (f"{track}  {domain} {data['comparisonsipc']} \n{cmpstr}\n")


            

opt20  rovers -18 
       decstar > delfi-blind    10 6 -> 6 6
       decstar > delfi-sccdfp    10 9 -> 6 6
       delfi-celmcut > decstar    12 10 -> 6 6
       delfi-celmcut > delfi-blind    12 6 -> 6 6
       delfi-celmcut > delfi-ipdb    12 10 -> 6 6
       delfi-celmcut > delfi-miasm    12 10 -> 6 6
       delfi-celmcut > delfi-sccdfp    12 9 -> 6 6
       delfi-celmcut > scorpion    12 11 -> 6 6
       delfi-ipdb > delfi-blind    10 6 -> 6 6
       delfi-ipdb > delfi-sccdfp    10 9 -> 6 6
       delfi-miasm > delfi-blind    10 6 -> 6 6
       delfi-miasm > delfi-sccdfp    10 9 -> 6 6
       delfi-sccdfp > delfi-blind    9 6 -> 6 6
       scorpion > decstar    11 10 -> 6 6
       scorpion > delfi-blind    11 6 -> 6 6
       scorpion > delfi-ipdb    11 10 -> 6 6
       scorpion > delfi-miasm    11 10 -> 6 6
       scorpion > delfi-sccdfp    11 9 -> 6 6

opt20  openstacks -17 
       decstar > delfi-blind    71 47 -> 3 3
       decstar > delfi-celmcut    71 44 -> 3 3
       decstar 